As usual we're start with cleaning and attempt to delay EDA until the end.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv('../data/train.csv')
df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '') #Just start with cleaning the titles
print(df.shape)
df.head()

#1.2 million people... no wonder this was such a big file. Realistic considering the industy and what I've worked with
#in the past.

#Recall that premium is this competion's target.

(1200000, 21)


,id,age,gender,annual_income,marital_status,number_of_dependents,education_level,occupation,health_score,location,policy_type,previous_claims,vehicle_age,credit_score,insurance_duration,policy_start_date,customer_feedback,smoking_status,exercise_frequency,property_type,premium_amount
0,0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0


In [3]:
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()])

Index(['age', 'annual_income', 'marital_status', 'number_of_dependents',
       'occupation', 'health_score', 'previous_claims', 'vehicle_age',
       'credit_score', 'insurance_duration', 'customer_feedback'],
      dtype='object')


In [4]:
#For kicks, I want to see if it'd be relevant to just keep 'pure' data:

df.dropna(inplace=False).shape[0], round(df.dropna(inplace=False).shape[0]/df.shape[0], 3)

#Yeah.... albeit we still have quite a large amount of data, and in most circumstances that's more than enough to work with,
#I want to demonstrate null handling for this, hopefully, portfolio-grade project. Might be fun later exploring the
#'pure' data: Noticing if the nulls were also consistent.... ie we had an even split of occupation in the overall data,
#which is not realistic in real life - does that trend remain in the 'pure' data [and then we may have to do oversampling]?

(384004, 0.32)

In [5]:
#After back and fourth with ChatGPT and I, I figured this approach is better. No need to loop.

thresh = .02

null_counts = pd.DataFrame({'null_sum': df.isnull().sum()})
null_counts['ratio'] = round(null_counts['null_sum']/df.shape[0], 3)
null_concerns = null_counts[null_counts['ratio']>thresh].sort_values('ratio', ascending=False)
null_concerns

#So, previous claims has quite a lot of nulls... and even occupation does. Interesting. I wonder if that's th same as unemployed.
#Credit score is always inteesting to see. Dependents are reasonable to ask, as well as feedback. Perhaps some similar issues iwth
#health score as well (ie on the surface I'd assume dubious data doesn't have a health score). No comment on income (in the
#context of nulls).

,null_sum,ratio
previous_claims,364029,0.303
occupation,358075,0.298
credit_score,137882,0.115
number_of_dependents,109672,0.091
customer_feedback,77824,0.065
health_score,74076,0.062
annual_income,44949,0.037


## Initial EDA

Note that this will be my first projec where I hopefully remember to get into the habit of pasting below into a bit of code to later preserve any changes made to the data to a cleaned csv.

Yeah, I'll keep it as is for now, howver next project I won't bother doing the various changes (null-filling, potential outlier trimming, etc.) until the very end.

In [6]:
i = -1

In [7]:
i += 1 #Eh...
print(f"Now looking at feature {i}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
df[df.columns[i]].value_counts(normalize=True)

Now looking at feature 0.


id
0          8.333333e-07
799988     8.333333e-07
800004     8.333333e-07
800003     8.333333e-07
800002     8.333333e-07
               ...     
399999     8.333333e-07
399998     8.333333e-07
399997     8.333333e-07
399996     8.333333e-07
1199999    8.333333e-07
Name: proportion, Length: 1200000, dtype: float64

In [8]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.") #Potentially redundant, but eh - I like to already see which
#feature it is. More intuitive and user-friendly.
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
#See below for commentary, but in short I decided to streamline this bit.
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 1, age.
Reminder that age has 18705 nulls (0.016%) - deal with them noob.
~~~
count    1.181295e+06
mean     4.114556e+01
std      1.353995e+01
min      1.800000e+01
25%      3.000000e+01
50%      4.100000e+01
75%      5.300000e+01
max      6.400000e+01
Name: age, dtype: float64


In [9]:
df['age'].fillna(df['age'].median(), inplace=True)

In [10]:
#Thanks to ChatGPT re. this neat bit right here.

pd.options.display.float_format = '{:,.2f}'.format
df['age'].describe()
# pd.reset_option('display.float_format')

count   1,200,000.00
mean           41.14
std            13.43
min            18.00
25%            30.00
50%            41.00
75%            53.00
max            64.00
Name: age, dtype: float64

Yeah, quite a nice distribution here. Shame it cuts off at 64. Unsurprising that it only starts at 18.

Now, immediately I'd want to band as we assume people of similar ages have similar enough characteristics. Furthermore, from an actuarial/rate perspective we'd do that all the time (for that or whatever other reasns). It definitely makes things simpler. I'm pretty sure even after banding we'd still consider specific ages.

So, starting from 18-22, simplifying all that to the '20' band.... and we'd likely either make the 60 band, consisting of people (by default) 58-62 to be bigger and also have 63 and 64... or a smaller 'old' band of anybody beyond that. Initially I was inclined a bigger final one, but my current desire is to have a smaller older band at our conclusion: It's flexible to account for a theortical change of data (that go beyond 64), retains spirit of my old work where we'd have a senior category, and it's funny (I plan on calling it 'oldies' or 'dinosaurs').

Anyways, essentially normally distributed around 41. Decent standard deviation, but makes sense given stuff (vague).

In [11]:
age_band_names = []
for band in range(20,65, 5):
    age_band_names.append(band)
# print(names)

for feature in age_band_names:
    cond1 = feature - 2
    cond2 = feature + 2
    df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]

#ChatGPT recommended something like this, but eh...
# for band in age_band_names[:-1]:  # Handle all except 'dinosaurs'
#     lower, upper = map(int, band.split('-'))
#     df[band] = ((df['age'] >= lower - 2) & (df['age'] <= upper + 2)).astype(int)
    
df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)#I'll keep their version. And they were right - why bother
#adding it earlier then have to deal with exception stuff if it's really just this...

In [12]:
pd.reset_option('display.float_format')

i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

#Nice even split. approximately.

Now looking at feature 2, gender.
~~~
gender
Male      0.502143
Female    0.497858
Name: proportion, dtype: float64


In [13]:
df['gender'] = [0 if gender == 'Female' else 1 for gender in df['gender']] #Recall that only a definite female
#we defined as male, with that gender abosrbing any 'other' - which would include an unknown. Especially in this data
#when it'd be the majority class.

In [14]:
pd.options.display.float_format = '{:,.2f}'.format

i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 3, annual_income.
Reminder that annual_income has 44949 nulls (0.037%) - deal with them noob.
~~~
count   1,155,051.00
mean       32,745.22
std        32,179.51
min             1.00
25%         8,001.00
50%        23,911.00
75%        44,634.00
max       149,997.00
Name: annual_income, dtype: float64


In [15]:
#Yeah, I should really keep the filling a seperate cell. Regardless. Let's analyze:
#Unexpectedly skewed to the right, with the median quite lower than the mean... Funny what year the date is based off of
#given inflation and the like. Regardless, the trend holds true.

#I'd expect a strong correlation to health, as by default if they can't even afford healthcare... At the same time, I recall
#the quandry of a certain streamer - if you're broke you get it for free and if you're rich it doesn't matter, but all the
#middl class.... And, evn thte upper tiers I could imagine depending upon their scenarios.... Ie even 150k is not all hat
#much in 2025....

df['annual_income'].fillna(df['annual_income'].median(), inplace=True) #Especially when either way I'd want to copy
#the code later on - I might as well get into the habit of using the precise name - as opposed to the variable is. Lest
#I mess up, want to change something, etc..

In [16]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 4, marital_status.
Reminder that marital_status has 18529 nulls (0.015%) - deal with them noob.
~~~
marital_status
Single     0.33
Married    0.33
Divorced   0.33
Name: proportion, dtype: float64


In [17]:
#Even enough distribution. Shame with the Pandas rounding as I'm sure it's not precise. Hmm, maybe I should just do both?
#So, in this case I'd assume single as definitionally they have themselves, so logically it would be the 'baseline'.

df['marital_status'].fillna('Single', inplace=True)

In [18]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 5, number_of_dependents.
Reminder that number_of_dependents has 109672 nulls (0.091%) - deal with them noob.
~~~
number_of_dependents
3.00   0.20
4.00   0.20
0.00   0.20
2.00   0.20
1.00   0.20
Name: proportion, dtype: float64


In [19]:
#We'll assume 0. Albeit not the median (2) I'd rather not add something when by default I can't prove its existence.
#I don't like this wording, but I'm trying to say: Unlike age where they must be something definitionally, there doesn't
#have to be dependents. Hence, without proof or any other additional considerations, I'd assume the default of 0.

df['number_of_dependents'].fillna(0, inplace=True)

In [20]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Likely because it's synthetic, but eh - I'm not liking these even splits.... Phds should be much smaller... Oh well.
#Reminder to dummify (forgot to do that). Not as relevant anymore as we're dummifying at the end.
#Oh, hence, what we discussed earlier: Might be a move to have that list. of objects.. oh well.

Now looking at feature 6, education_level.
~~~
education_level
Master's      0.25
PhD           0.25
Bachelor's    0.25
High School   0.24
Name: proportion, dtype: float64


In [21]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 7, occupation.
Reminder that occupation has 358075 nulls (0.298%) - deal with them noob.
~~~
occupation
Employed        0.34
Self-Employed   0.34
Unemployed      0.33
Name: proportion, dtype: float64


In [22]:
#Like number of dependents, I'll assume no job.
df['occupation'].fillna('Unemployed', inplace=True)

In [23]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")

#Tangential, but a bit tempted to try something like this:
#I'm getting tired of having to rechange the code dep. on the type of feature.
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']: #Arb. threshold
    print(df[df.columns[i]].describe()) #note that display could be another option.
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 8, health_score.
Reminder that health_score has 74076 nulls (0.062%) - deal with them noob.
~~~
count   1,125,924.00
mean           25.61
std            12.20
min             2.01
25%            15.92
50%            24.58
75%            34.53
max            58.98
Name: health_score, dtype: float64


Tangential (again) but I wonder if for EDA such as this it might be better to just make an initia split of (suspected) categorical and numeric variables and go down with them. Eh... we'll see.

Off hand I don't recall a definition of healthscare. However, the point is clear.

In [24]:
df['health_score'].fillna(df['health_score'].median(), inplace=True)

In [25]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
#Seriously, these splits are waaaay too even. One thing if this would be classification and we're talking about our targt,
#but for the features.... and every single one? I supposeit is ideal, but I don't want that in my data!!!

#Until data gets interesting I won't bother to comment. Ie even if realistically the groups should be skewed in whatever way,
#like expecting a minority of Phds...

Now looking at feature 9, location.
~~~
location
Suburban   0.33
Rural      0.33
Urban      0.33
Name: proportion, dtype: float64


In [26]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Would expect and hope premium pays more. Potentialy could make these hierarchical. But, for now...

Now looking at feature 10, policy_type.
~~~
policy_type
Premium         0.33
Comprehensive   0.33
Basic           0.33
Name: proportion, dtype: float64


In [27]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 11, previous_claims.
Reminder that previous_claims has 364029 nulls (0.303%) - deal with them noob.
~~~
previous_claims
0.00   0.37
1.00   0.36
2.00   0.20
3.00   0.06
4.00   0.01
5.00   0.00
6.00   0.00
7.00   0.00
8.00   0.00
9.00   0.00
Name: proportion, dtype: float64


In [28]:
#Makes sense to see such a skew, as unlikely for people to have more and more.
#Note that the max is actually 9 (well, we know that from our code logic already):
print(df['previous_claims'].describe())
atleast_1 = sum(df['previous_claims']>0)
print(df.shape[0] - atleast_1, atleast_1)

#Like before with dependents. See our logic there.
df['previous_claims'].fillna(0, inplace=True)

count   835,971.00
mean          1.00
std           0.98
min           0.00
25%           0.00
50%           1.00
75%           2.00
max           9.00
Name: previous_claims, dtype: float64
669462 530538


Wow what a mean considering all the 0s. Guess it's not that surprising as the 0s and 1s average out to be .5 then consider all the rest... Regardless, let's see (and comment in this block) if it's at least one:

Yeah, then this is quite interesting. Now we see a a hefty skew to the right (before the opposite, albeit not as exagerated). Ie you got those 'unhealthy' people always getting sick. Potentially they didn't get enough care. Regardless, the regulars dominate the activity here.

In [29]:
df[df['previous_claims']>0]['previous_claims'].describe()

count   530,538.00
mean          1.58
std           0.78
min           1.00
25%           1.00
50%           1.00
75%           2.00
max           9.00
Name: previous_claims, dtype: float64

In [30]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 12, vehicle_age.
Reminder that vehicle_age has 6 nulls (0.0%) - deal with them noob.
~~~
count   1,199,994.00
mean            9.57
std             5.78
min             0.00
25%             5.00
50%            10.00
75%            15.00
max            19.00
Name: vehicle_age, dtype: float64


In [31]:
#Bit to the lft, but overall nice distribution. Honestly should be expected with this dataset by now...

#Of note are these babies - notice how we have only whole numbers so likly they're just very new cars.

sum(df['vehicle_age']==0), round(sum(df['vehicle_age']==0)/df.shape[0], 3)

(61615, 0.051)

In [32]:
df['vehicle_age'].fillna(df['vehicle_age'].median(), inplace=True) #Like human age must be something. In this case with so
#Few nulls though a bit negligible.

In [33]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Again, another clean even (ess.) dist.. Recall that the max is 300 and absolute is 850..

Now looking at feature 13, credit_score.
Reminder that credit_score has 137882 nulls (0.115%) - deal with them noob.
~~~
count   1,062,118.00
mean          592.92
std           149.98
min           300.00
25%           468.00
50%           595.00
75%           721.00
max           849.00
Name: credit_score, dtype: float64


In [34]:
df['credit_score'].fillna(df['credit_score'].median(), inplace=True)

In [35]:
#Just for kicks, I want to see how evenly distributed some of the variables are:
df[df['credit_score']<350]['education_level'].value_counts(normalize=True)

education_level
Master's      0.26
PhD           0.25
Bachelor's    0.25
High School   0.24
Name: proportion, dtype: float64

In [36]:
df[df['credit_score']>800]['education_level'].value_counts(normalize=True)
#Unsurprising....

education_level
PhD           0.26
Bachelor's    0.25
Master's      0.25
High School   0.24
Name: proportion, dtype: float64

In [37]:
##We now fixed this, but regardless:

# Oh gf me - forgot to dummify this stuff. Given this last bit of code it might be better to dummify at the end.
# Or, do something like keeping the original. Do my EDA, then redo the dummification/actually do it at the end.
# Especially when either way we'd need to do so for the test data we might as well not get ourselves confused with that (
# recall the +1 potential disasters of our i-scrolling) & avoid this issue [easily compare all of a feature's categories
# together].

# From ChatGPT I'll sort out later.... Enough of the coding and analysis practice for today.

# df['occupation_original'] = df['occupation']  # Preserve original column
# df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)

In [38]:
# original_col = df['occupation']
# df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)
# df.insert(0, 'occupation_original', original_col)  # Insert at the start

In [39]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Unsure if this is in months or years. It also just dawned on me - was this for car data? Would makee sense given the vehicle
#age feature. Just assumed it was health. Oh well - all the ideas remain the same....

#Hmm, I double chcked th page and I'm honestly not convinced from the page. Awkward... Inclined to still assume health/life though.
#Now, does it matter? Potentially not. But that's a discussion for later (likely when we get to premium).

Now looking at feature 14, insurance_duration.
Reminder that insurance_duration has 1 nulls (0.0%) - deal with them noob.
~~~
insurance_duration
9.00   0.11
1.00   0.11
8.00   0.11
7.00   0.11
5.00   0.11
4.00   0.11
6.00   0.11
3.00   0.11
2.00   0.11
Name: proportion, dtype: float64


In [40]:
df['insurance_duration'].describe() #Ooh quite a nice distribution. Seriously though, these are all way too organized...

count   1,199,999.00
mean            5.02
std             2.59
min             1.00
25%             3.00
50%             5.00
75%             7.00
max             9.00
Name: insurance_duration, dtype: float64

In [41]:
#Just one null - replacing it with the median which is approx. the mean...
df['insurance_duration'].fillna(5, inplace=True)

In [42]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Yeah, we gotta clean the times. Or at least do something (potentially ignore...) with them. For now let's do the easy stuff.
#To at least analyze: Month might matter. Day of week/weekend might matter. Time might mater...

Now looking at feature 15, policy_start_date.
~~~
policy_start_date
2020-02-08 15:21:39.134960   0.00
2023-08-13 15:21:39.155231   0.00
2022-02-02 15:21:39.134960   0.00
2022-08-30 15:21:39.134960   0.00
2023-11-02 15:21:39.134960   0.00
                             ... 
2021-06-07 15:21:39.104139   0.00
2024-07-19 15:21:39.233998   0.00
2019-12-14 15:21:39.110557   0.00
2020-07-23 15:21:39.217387   0.00
2020-10-19 15:21:39.118178   0.00
Name: proportion, Length: 167381, dtype: float64


In [43]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
    
#Unlike education level, likely a lot more justified to map these two numbers: -1, 0, 1 seems the most logical to me.
#Still might be worthwhile to dummify like the rest; but I won't.
#Ie with education I don't think the advantages of increasing the education are linear (rather diminishing returns after
#bachelors).

Now looking at feature 16, customer_feedback.
Reminder that customer_feedback has 77824 nulls (0.065%) - deal with them noob.
~~~
customer_feedback
Average   0.34
Poor      0.33
Good      0.33
Name: proportion, dtype: float64


In [44]:
df['customer_feedback'].fillna('Average', inplace=True)
#From ChatGPT for convenience:

mapping = {
    'Poor': -1,
    'Average': 0,
    'Good': 1
}
df['customer_feedback'] = df['customer_feedback'].map(mapping)

In [45]:
df['customer_feedback'].value_counts(normalize=True)

customer_feedback
 0   0.38
-1   0.31
 1   0.31
Name: proportion, dtype: float64

In [46]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
#Come on - even by the smokers?!?!? Perhaps weed, heh.
#Onn the topic that's actualyl quite interesting - smoking what? Historically would be cigarettes, but in 2025 likely th
#percenages of 'smokers' and their relativ changes in health vary quite a lot from drug to drug.

Now looking at feature 17, smoking_status.
~~~
smoking_status
Yes   0.50
No    0.50
Name: proportion, dtype: float64


In [47]:
df['smoking_status'] = [0 if status == 'No' else 1 for status in df['smoking_status']]

In [48]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 18, exercise_frequency.
~~~
exercise_frequency
Weekly    0.26
Monthly   0.25
Rarely    0.25
Daily     0.25
Name: proportion, dtype: float64


Hmm, likely this should be dummified as I venture the health changes are definitely not linear.

In [49]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))
#Perhaps another proxy of stability and healtth.

Now looking at feature 19, property_type.
~~~
property_type
House       0.33
Apartment   0.33
Condo       0.33
Name: proportion, dtype: float64


In [50]:
i += 1
print(f"Now looking at feature {i}, {df.columns[i]}.")
nulls = df[df.columns[i]].isnull().sum()
if nulls > 0:
    print(f"Reminder that {df.columns[i]} has {nulls} nulls ({round(nulls/df.shape[0], 3)}%) - deal with them noob.")
print('~~~')
if len(df[df.columns[i]].value_counts()) > 10 and df[df.columns[i]].dtype in ['int64', 'float64']:
    print(df[df.columns[i]].describe())
else:
    print(df[df.columns[i]].value_counts(normalize=True))

Now looking at feature 20, premium_amount.
~~~
count   1,200,000.00
mean        1,102.54
std           865.00
min            20.00
25%           514.00
50%           872.00
75%         1,509.00
max         4,999.00
Name: premium_amount, dtype: float64


And now our long awaited target - premium.

Nice to finally see a non-normal distrubtion. Notice th hefty skew to the right of quite a larger mean than median. Furthermor, look at that standard deviation - almost as large as the median itself! So, unsurprisingly we see a few people paying a lot.

In [51]:
#Conf. that I got them all.
if sum(df.isnull().sum())==0:
    print("Phew, no nulls.")
else:
    print(df.columns[df.isnull().any()])

Phew, no nulls.


### Re. time:

I think I'll just ignore for now; no noob they might be super-charged if they didn't sign up during open enrollment. Don't be lazy and at least prepare the data - you never have to actually use it... Fine.

In [53]:
df['policy_start_date'].dtype

dtype('O')

In [55]:
df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])
df['policy_start_date'].dtype

dtype('<M8[ns]')

In [56]:
#policy_start_date
df['psd_year'] = df['policy_start_date'].dt.year
df['psd_month'] = df['policy_start_date'].dt.month
df['psd_day'] = df['policy_start_date'].dt.day
#Eh, we'll end it at day.

In [60]:
df.head()[['policy_start_date', 'psd_year', 'psd_month', 'psd_day']]

,policy_start_date,psd_year,psd_month,psd_day
0,2023-12-23 15:21:39.134960,2023,12,23
1,2023-06-12 15:21:39.111551,2023,6,12
2,2023-09-30 15:21:39.221386,2023,9,30
3,2024-06-12 15:21:39.226954,2024,6,12
4,2021-12-01 15:21:39.252145,2021,12,1


In [ ]:
#Per customization/feature-engineering we'll also do if it's a weekend or not. We won't bother gettinginto national holidays,
#assuming this is in America.

In [61]:
df['psd_day_of_week'] = df['policy_start_date'].dt.day_name()
df.head()[['policy_start_date', 'psd_day_of_week']]

,policy_start_date,psd_day_of_week
0,2023-12-23 15:21:39.134960,Saturday
1,2023-06-12 15:21:39.111551,Monday
2,2023-09-30 15:21:39.221386,Saturday
3,2024-06-12 15:21:39.226954,Wednesday
4,2021-12-01 15:21:39.252145,Wednesday


In [62]:
df['psd_day_of_week'].value_counts(normalize=True)#Wow, they even got it even here... Note that I definitely doubt this in
#real life.... Even more than the phd distribution - aint nobody want to work on the weekend! #Loverboy, heh

psd_day_of_week
Wednesday   0.14
Monday      0.14
Tuesday     0.14
Sunday      0.14
Saturday    0.14
Thursday    0.14
Friday      0.14
Name: proportion, dtype: float64

In [63]:
df['psd_year'].value_counts(normalize=True) #Bit more of a change here. We ake it!

psd_year
2022   0.20
2021   0.20
2020   0.20
2023   0.20
2024   0.12
2019   0.07
Name: proportion, dtype: float64

In [64]:
df['psd_month'].value_counts(normalize=True) #Even

psd_month
5    0.09
3    0.09
8    0.08
4    0.08
7    0.08
11   0.08
10   0.08
9    0.08
1    0.08
6    0.08
12   0.08
2    0.08
Name: proportion, dtype: float64

In [67]:
df['psd_day'].value_counts(normalize=True).sort_values(ascending=True)[:5] #""

psd_day
31   0.02
30   0.03
10   0.03
28   0.03
29   0.03
Name: proportion, dtype: float64

In [68]:
#Hmm, so let's dummify the days. I'll leave the f.engineering at if it's a weekend though.
weekend = ['Saturday', 'Sunday']
df['psd_weekend'] = [1 if day in weekend else 0 for day in df['psd_day_of_week']]

In [ ]:
#Dummy section, as we want to keep th categories as strings/objects at the moment for easier eda

# df = pd.get_dummies(df, columns=['marital_status'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['education_level'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['location'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['policy_type'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['exercise_frequency'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['property_type'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['psd_day_of_week'], dtype=int, drop_first=True) #Hmm, in this case a bit more inclined to actually
#not drop the first. Perhaps because intuitivlely day_x is easier to compare than say education_x vs. anything else.
##ManicMondays #HavingFunWithMyReferences

In [71]:
print(df.shape)
df.head() #Hmm, is it a move to rearrange the target to the end? Oh well.

(1200000, 36)


,id,age,gender,annual_income,marital_status,number_of_dependents,education_level,occupation,health_score,location,policy_type,previous_claims,vehicle_age,credit_score,insurance_duration,policy_start_date,customer_feedback,smoking_status,exercise_frequency,property_type,premium_amount,20,25,30,35,40,45,50,55,60,dinosaurs,psd_year,psd_month,psd_day,psd_day_of_week,psd_weekend
0,0,19.00,0,"10,049.00",Married,1.00,Bachelor's,Self-Employed,22.60,Urban,Premium,2.00,17.00,372.00,5.00,2023-12-23 15:21:39.134960,-1,0,Weekly,House,"2,869.00",1,0,0,0,0,0,0,0,0,0,2023,12,23,Saturday,1
1,1,39.00,0,"31,678.00",Divorced,3.00,Master's,Unemployed,15.57,Rural,Comprehensive,1.00,12.00,694.00,2.00,2023-06-12 15:21:39.111551,0,1,Monthly,House,"1,483.00",0,0,0,0,1,0,0,0,0,0,2023,6,12,Monday,0
2,2,23.00,1,"25,602.00",Divorced,3.00,High School,Self-Employed,47.18,Suburban,Premium,1.00,14.00,595.00,3.00,2023-09-30 15:21:39.221386,1,1,Weekly,House,567.00,0,1,0,0,0,0,0,0,0,0,2023,9,30,Saturday,1
3,3,21.00,1,"141,855.00",Married,2.00,Bachelor's,Unemployed,10.94,Rural,Basic,1.00,0.00,367.00,1.00,2024-06-12 15:21:39.226954,-1,1,Daily,Apartment,765.00,1,0,0,0,0,0,0,0,0,0,2024,6,12,Wednesday,0
4,4,21.00,1,"39,651.00",Single,1.00,Bachelor's,Self-Employed,20.38,Rural,Premium,0.00,8.00,598.00,4.00,2021-12-01 15:21:39.252145,-1,1,Weekly,House,"2,022.00",1,0,0,0,0,0,0,0,0,0,2021,12,1,Wednesday,0


## Concluding Matters

Hmm, from an efficiency standpoint I wonder if it's better to not even change the original data and just post here, even for the train data, all the various changes - just once.

Yeah, as I decided earlier, we'll try to keep analysis separate next time and just have all the various changes we do to
clean the data at the end. At least for now (2/13/25) I'll be leaving the code as is.

In [78]:
# #As I said earlier, prepping the total changed cleanings for modeling.
# #Ie kind of like on Git we have the commit before actually pushing.
# df = pd.read_csv('../data/train.csv')
# df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')
# for col in df.columns:
#     try:
#         df[col] = round(df[col], 3)
#     except:
#         pass

# df['age'].fillna(df['age'].median(), inplace=True)

# age_band_names = []
# for band in range(20,65, 5):
#     age_band_names.append(band)
# for feature in age_band_names:
#     cond1 = feature - 2
#     cond2 = feature + 2
#     df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]
# df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)

# df['gender'] = [0 if gender == 'Female' else 1 for gender in df['gender']]

# df['annual_income'].fillna(df['annual_income'].median(), inplace=True)

# df['marital_status'].fillna('Single', inplace=True)

# df['number_of_dependents'].fillna(0, inplace=True)

# df['occupation'].fillna('Unemployed', inplace=True)

# df['health_score'].fillna(df['health_score'].median(), inplace=True)

# df['previous_claims'].fillna(0, inplace=True)

# df['vehicle_age'].fillna(df['vehicle_age'].median(), inplace=True)

# df['credit_score'].fillna(df['credit_score'].median(), inplace=True)

# df['insurance_duration'].fillna(5, inplace=True)

# df['customer_feedback'].fillna('Average', inplace=True)
# mapping = {
#     'Poor': -1,
#     'Average': 0,
#     'Good': 1
# }
# df['customer_feedback'] = df['customer_feedback'].map(mapping)

# df['smoking_status'] = [0 if status == 'No' else 1 for status in df['smoking_status']]

# df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])
# df['psd_year'] = df['policy_start_date'].dt.year
# df['psd_month'] = df['policy_start_date'].dt.month
# df['psd_day'] = df['policy_start_date'].dt.day
# df['psd_day_of_week'] = df['policy_start_date'].dt.day_name()
# weekend = ['Saturday', 'Sunday']
# df['psd_weekend'] = [1 if day in weekend else 0 for day in df['psd_day_of_week']]

# if sum(df.isnull().sum())==0:
#     print("Phew, no nulls.")
# else:
#     print(df.columns[df.isnull().any()])

# #Dummies now:
# df = pd.get_dummies(df, columns=['marital_status'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['education_level'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['location'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['policy_type'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['exercise_frequency'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['property_type'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['psd_day_of_week'], dtype=int, drop_first=True)

# print(df.shape)

Phew, no nulls.
(1200000, 50)


In [75]:
# df.to_csv('../data/cleaned_train.csv', index=False)

In [76]:
# df = pd.read_csv('../data/test.csv')
# df.columns = df.columns.str.lower().str.replace(' ', '_').str.replace('.', '')
# for col in df.columns:
#     try:
#         df[col] = round(df[col], 3)
#     except:
#         pass

# df['age'].fillna(df['age'].median(), inplace=True)

# age_band_names = []
# for band in range(20,65, 5):
#     age_band_names.append(band)
# for feature in age_band_names:
#     cond1 = feature - 2
#     cond2 = feature + 2
#     df[str(feature)] = [1 if (age >= cond1 and age <= cond2) else 0 for age in df['age']]
# df['dinosaurs'] = (df['age'] > age_band_names[-1]+2).astype(int)

# df['gender'] = [0 if gender == 'Female' else 1 for gender in df['gender']]

# df['annual_income'].fillna(df['annual_income'].median(), inplace=True)

# df['marital_status'].fillna('Single', inplace=True)

# df['number_of_dependents'].fillna(0, inplace=True)

# df['occupation'].fillna('Unemployed', inplace=True)

# df['health_score'].fillna(df['health_score'].median(), inplace=True)

# df['previous_claims'].fillna(0, inplace=True)

# df['vehicle_age'].fillna(df['vehicle_age'].median(), inplace=True)

# df['credit_score'].fillna(df['credit_score'].median(), inplace=True)

# df['insurance_duration'].fillna(5, inplace=True)

# df['customer_feedback'].fillna('Average', inplace=True)
# mapping = {
#     'Poor': -1,
#     'Average': 0,
#     'Good': 1
# }
# df['customer_feedback'] = df['customer_feedback'].map(mapping)

# df['smoking_status'] = [0 if status == 'No' else 1 for status in df['smoking_status']]

# df['policy_start_date'] = pd.to_datetime(df['policy_start_date'])
# df['psd_year'] = df['policy_start_date'].dt.year
# df['psd_month'] = df['policy_start_date'].dt.month
# df['psd_day'] = df['policy_start_date'].dt.day
# df['psd_day_of_week'] = df['policy_start_date'].dt.day_name()
# weekend = ['Saturday', 'Sunday']
# df['psd_weekend'] = [1 if day in weekend else 0 for day in df['psd_day_of_week']]

# if sum(df.isnull().sum())==0:
#     print("Phew, no nulls.")
# else:
#     print(df.columns[df.isnull().any()])

# #Dummies now:
# df = pd.get_dummies(df, columns=['marital_status'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['education_level'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['occupation'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['location'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['policy_type'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['exercise_frequency'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['property_type'], dtype=int, drop_first=True)
# df = pd.get_dummies(df, columns=['psd_day_of_week'], dtype=int, drop_first=True)

# print(df.shape) #I'm curious. Pre-gaming the train. By the way - we did all our eda on just the train - AND NOT the test.
# #So, obviously I am assuming that the data is comprababl. And, I hope hta is reasonable... Eh, let us continue.

# #Honestly, if this would be a formal business problem I would check. Perhaps could write some code to do so. Ie if the means of the
# #various features are  close enough...

Phew, no nulls.
(800000, 49)


## Hold Up

Wait, we have 49 features there?? OH yeah - recall our last shape as before that. From quick math: It probably adds up. Well, guess we'll see later.

Eh don't be lazy and re run it - Hmmmmmm 50 vs. 49.... Oh yeah - no premium! Mmk so we're good to go.

In [77]:
# df.to_csv('../data/cleaned_test.csv', index=False)